In [2]:
from ultralytics import YOLO

# --- 1. Cargar el modelo base yolo11n.pt ---
# Asumiendo que tienes este archivo y es compatible.
try:
    model = YOLO('yolo11n.pt') # Carga los pesos base de yolo11n
    print("Modelo base 'yolo11n.pt' cargado correctamente.")
except Exception as e:
    print(f"Error cargando el modelo base 'yolo11n.pt': {e}")
    print("Asegúrate de que el archivo existe y es compatible con Ultralytics.")
    exit()

# --- 2. Iniciar Entrenamiento ---
# Entrena el modelo usando tu archivo 'matriculas_solo.yaml'
# Este YAML DEBE tener nc: 1 y names: ['matricula']
results = model.train(
    data='matriculas.yaml',    # Ruta al archivo YAML (solo 1 clase: matricula)
    epochs=70,                    # Número de épocas (ajusta según necesites)
    imgsz=640,                    # Tamaño de imagen
    batch=4,                     # Tamaño del lote
    name='modelo_matriculas_yolo11', # Nombre para la carpeta de resultados
    exist_ok=True                 # Permite sobrescribir
)

print("\n¡Entrenamiento para SOLO matrículas (base yolo11n) completado!")
# La ruta real donde se guardan los pesos
print(f"Mejores pesos del modelo guardados en: runs/detect/{results.save_dir}/weights/best.pt")

Modelo base 'yolo11n.pt' cargado correctamente.
New https://pypi.org/project/ultralytics/8.3.223 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.215  Python-3.9.23 torch-2.8.0+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=matriculas.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=70, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=mo

In [3]:
from ultralytics import YOLO
import cv2
import glob
import os
import matplotlib.pyplot as plt

# --- CONFIGURACIÓN DE MODELOS, dibujar imagen ---

# Carga el modelo general YOLOv8 pre-entrenado (para vehículos)
try:
    model_general_coches = YOLO('yolov8n.pt') # Usamos yolov8n estándar para coches
    print("Modelo general (yolov8n.pt) para coches cargado correctamente.")
except Exception as e:
    print(f"Error cargando modelo general 'yolov8n.pt': {e}")
    exit()

# Carga TU modelo entrenado (el que se basó en yolo11n.pt)
# ¡¡IMPORTANTE!! Ajusta esta ruta al 'best.pt' del entrenamiento anterior.
try:
    # Usa el nombre que le diste al entrenamiento ('modelo_matriculas_yolo11')
    ruta_modelo_matriculas_entrenado = 'runs/detect/modelo_matriculas_yolo11/weights/best.pt' # <--- AJUSTA ESTA RUTA
    model_matriculas = YOLO(ruta_modelo_matriculas_entrenado)
    print(f"Modelo entrenado para matrículas '{ruta_modelo_matriculas_entrenado}' cargado correctamente.")
except Exception as e:
    print(f"Error cargando tu modelo entrenado de matrículas '{ruta_modelo_matriculas_entrenado}': {e}")
    print("Asegúrate de que la ruta apunta al archivo 'best.pt' correcto.")
    exit()

# Clases de interés del modelo general (vehículos COCO)
coco_vehicle_classes = [2, 3, 5, 7] # car, motorcycle, bus, truck

# Colores y confianzas
color_vehiculo = (255, 0, 0) # Azul
color_matricula = (0, 255, 0) # Verde
conf_vehiculo = 0.40
conf_matricula = 0.1

# --- DEFINIR IMÁGENES DE PRUEBA Y SALIDA ---
ruta_carpeta_imagenes_test = './TGC_RBNW/imgenes_comprobar/*'
rutas_imagenes_test = glob.glob(ruta_carpeta_imagenes_test)
if not rutas_imagenes_test:
    print(f"Error: No se encontraron imágenes en '{ruta_carpeta_imagenes_test}'.")
    exit()
output_dir_deteccion_combinada = 'resultados2_deteccion_combinada_y11base'
os.makedirs(output_dir_deteccion_combinada, exist_ok=True)

print(f"Procesando {len(rutas_imagenes_test)} imágenes...")
print(f"Resultados guardados en: {output_dir_deteccion_combinada}")

# --- PROCESAR IMÁGENES Y DIBUJAR ---
for img_path in rutas_imagenes_test:
    base_filename = os.path.basename(img_path)
    # print(f"Procesando: {base_filename}") # Puedes descomentar para ver el progreso

    frame = cv2.imread(img_path)
    if frame is None: continue
    frame_annotated = frame.copy()

    # --- 1. Detectar VEHÍCULOS ---
    results_coches = model_general_coches.predict(frame, classes=coco_vehicle_classes, conf=conf_vehiculo, verbose=False)
    if len(results_coches) > 0 and len(results_coches[0].boxes) > 0:
        for box_veh in results_coches[0].boxes:
            coords_veh = box_veh.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords_veh
            conf_v = box_veh.conf.cpu().numpy()[0]
            class_id_v = int(box_veh.cls.cpu().numpy()[0])
            label_veh = model_general_coches.names[class_id_v]
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), color_vehiculo, 2)
            cv2.putText(frame_annotated, f'{label_veh} {conf_v:.2f}', (x1_v, y1_v - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_vehiculo, 2)

    # --- 2. Detectar MATRÍCULAS ---
    results_matricula = model_matriculas.predict(frame, conf=conf_matricula, verbose=False)
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        for box_mat in results_matricula[0].boxes:
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            conf_m = box_mat.conf.cpu().numpy()[0]
            label_mat = model_matriculas.names[0] # Siempre será 'matricula'
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), color_matricula, 2)
            cv2.putText(frame_annotated, f'{label_mat} {conf_m:.2f}', (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_matricula, 2)

    # --- GUARDAR ---
    output_path = os.path.join(output_dir_deteccion_combinada, f"comb_{base_filename}")
    cv2.imwrite(output_path, frame_annotated)

# --- FINALIZACIÓN ---
print("\nProceso de detección combinada completado.")
print(f"Resultados guardados en: {output_dir_deteccion_combinada}")

Modelo general (yolov8n.pt) para coches cargado correctamente.
Modelo entrenado para matrículas 'runs/detect/modelo_matriculas_yolo11/weights/best.pt' cargado correctamente.
Procesando 4 imágenes...
Resultados guardados en: resultados2_deteccion_combinada_y11base

Proceso de detección combinada completado.
Resultados guardados en: resultados2_deteccion_combinada_y11base


In [6]:
from ultralytics import YOLO
import cv2
import glob
import os
import matplotlib.pyplot as plt
import easyocr  # <-- NUEVO: Importar easyOCR
import time     # <-- NUEVO: Para medir tiempos de inferencia

# --- CONFIGURACIÓN DE MODELOS ---

# Carga el modelo general YOLOv8 (para vehículos)
try:
    model_general_coches = YOLO('yolov8n.pt')
    print("Modelo general (yolov8n.pt) para coches cargado correctamente.")
except Exception as e:
    print(f"Error cargando modelo general 'yolov8n.pt': {e}")
    exit()

# Carga TU modelo entrenado (matrículas)
try:
    ruta_modelo_matriculas_entrenado = 'runs/detect/modelo_matriculas_yolo11/weights/best.pt'
    model_matriculas = YOLO(ruta_modelo_matriculas_entrenado)
    print(f"Modelo entrenado para matrículas '{ruta_modelo_matriculas_entrenado}' cargado correctamente.")
except Exception as e:
    print(f"Error cargando tu modelo entrenado de matrículas: {e}")
    exit()

# --- NUEVO: Inicializar el lector de easyOCR ---
# Esto carga el modelo de OCR en memoria. Hacemos esto FUERA del bucle.
# 'es' para español, 'en' para inglés (bueno para letras/números estándar)
try:
    reader = easyocr.Reader(['es', 'en'], gpu=False) # Pon gpu=True si tienes NVIDIA y CUDA
    print("easyOCR cargado correctamente.")
except Exception as e:
    print(f"Error cargando easyOCR: {e}")
    exit()


# ... (tu código de clases, colores y rutas de test se mantiene igual) ...
coco_vehicle_classes = [2, 3, 5, 7] 
color_vehiculo = (255, 0, 0)
color_matricula = (0, 255, 0)
conf_vehiculo = 0.40
conf_matricula = 0.1 # Dejamos la confianza baja que funcionó

ruta_carpeta_imagenes_test = './TGC_RBNW/imgenes_comprobar/*' # <-- AJUSTA SI USAS EL NUEVO DATASET
rutas_imagenes_test = glob.glob(ruta_carpeta_imagenes_test)
output_dir_deteccion_combinada = 'resultados_con_OCR' # <-- He cambiado el nombre
os.makedirs(output_dir_deteccion_combinada, exist_ok=True)

print(f"Procesando {len(rutas_imagenes_test)} imágenes...")

# --- PROCESAR IMÁGENES Y DIBUJAR ---

for img_path in rutas_imagenes_test:
    base_filename = os.path.basename(img_path)
    frame = cv2.imread(img_path)
    if frame is None: continue
    frame_annotated = frame.copy() # Copia para dibujar

    # 1. Detectar VEHÍCULOS (sin cambios)
    results_coches = model_general_coches.predict(frame, classes=coco_vehicle_classes, conf=conf_vehiculo, verbose=False)
    if len(results_coches) > 0 and len(results_coches[0].boxes) > 0:
        for box_veh in results_coches[0].boxes:
             # ... (todo tu código para dibujar el rectángulo azul del coche) ...
            coords_veh = box_veh.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords_veh
            label_veh = model_general_coches.names[int(box_veh.cls.cpu().numpy()[0])]
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), color_vehiculo, 2)
            cv2.putText(frame_annotated, f'{label_veh}', (x1_v, y1_v - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_vehiculo, 2)


    # --- 2. Detectar MATRÍCULAS (¡CON CAMBIOS!) ---
    results_matricula = model_matriculas.predict(frame, conf=conf_matricula, verbose=False)
    
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        for box_mat in results_matricula[0].boxes:
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            conf_m = box_mat.conf.cpu().numpy()[0]
            
            # --- INICIO DE LÓGICA OCR ---
            
            # Recortar la matrícula de la imagen ORIGINAL
            # Añadimos un pequeño margen (padding) de 5px si es posible
            my1_pad = max(0, my1 - 5)
            my2_pad = min(frame.shape[0], my2 + 5)
            mx1_pad = max(0, mx1 - 5)
            mx2_pad = min(frame.shape[1], mx2 + 5)
            matricula_recortada = frame[my1_pad:my2_pad, mx1_pad:mx2_pad]

            # Inicializar texto
            texto_matricula = "N/A"
            
            # Medir tiempo de OCR
            t_start_ocr = time.time()
            
            try:
                # Usar easyOCR para leer el texto del RECORTE
                # detail=0 significa que solo devuelve el texto, no las coordenadas
                ocr_result = reader.readtext(matricula_recortada, detail=0, allowlist='0123456789ABCDEFGHIJKLMNPQRSTUVWXYZ') # Whitelist para matrículas
                
                if ocr_result:
                    texto_matricula = ocr_result[0].upper().replace(" ", "") # Limpiar texto
                    
            except Exception as e:
                print(f"Error en OCR: {e}")
                
            t_end_ocr = time.time()
            tiempo_ocr = (t_end_ocr - t_start_ocr) * 1000 # en milisegundos
            
            # --- FIN DE LÓGICA OCR ---

            # Dibujar el rectángulo de la matrícula
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), color_matricula, 2)
            
            # Escribir el TEXTO de la matrícula encima
            label_final = f'{texto_matricula} ({conf_m:.2f})'
            cv2.putText(frame_annotated, label_final, (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_matricula, 2)
            
            print(f"  > {base_filename}: Encontrada matrícula: {texto_matricula} (Conf: {conf_m:.2f}) (Tiempo OCR: {tiempo_ocr:.1f} ms)")
            # Aquí guardarías 'texto_matricula' para tu CSV
            
            # --- FIN DE LÓGICA OCR ---


    # --- GUARDAR ---
    output_path = os.path.join(output_dir_deteccion_combinada, f"ocr_{base_filename}")
    cv2.imwrite(output_path, frame_annotated)

print("\nProceso de detección con OCR completado.")

Using CPU. Note: This module is much faster with a GPU.


Modelo general (yolov8n.pt) para coches cargado correctamente.
Modelo entrenado para matrículas 'runs/detect/modelo_matriculas_yolo11/weights/best.pt' cargado correctamente.
easyOCR cargado correctamente.
Procesando 4 imágenes...
  > img_comprobar1.jpg: Encontrada matrícula: 6628GXR (Conf: 0.95) (Tiempo OCR: 223.1 ms)
  > img_comprobar1.jpg: Encontrada matrícula: 15532BYP (Conf: 0.30) (Tiempo OCR: 193.5 ms)
  > img_comprobar2.jpg: Encontrada matrícula: JPGC7085N (Conf: 0.84) (Tiempo OCR: 185.2 ms)
  > img_comprobar3.jpg: Encontrada matrícula: N/A (Conf: 0.20) (Tiempo OCR: 34.5 ms)
  > img_comprobar4.jpg: Encontrada matrícula: JNV (Conf: 0.91) (Tiempo OCR: 233.1 ms)
  > img_comprobar4.jpg: Encontrada matrícula: 3232JCJ (Conf: 0.57) (Tiempo OCR: 110.5 ms)
  > img_comprobar4.jpg: Encontrada matrícula: N/A (Conf: 0.18) (Tiempo OCR: 64.1 ms)

Proceso de detección con OCR completado.
